In [8]:
import pandas as pd
import numpy as np
import json
import gzip
from gensim.parsing.preprocessing import remove_stopwords
import gensim
import os
import time
from datetime import datetime
data_dir = "data/Garden"

In [24]:
def parse(path):
    g = gzip.open(path, "rb")
    for l in g:
        yield eval(l)

def load_data(file_name, head = 500):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l.decode('utf-8'))
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
#             if (head is not None) and (count > head):
#                 break
    return data
        
def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [25]:
data = load_data("data/book/goodreads_books_poetry.json.gz")
data = pd.DataFrame(data)

data.head()

,asin,authors,average_rating,book_id,country_code,description,edition_information,format,image_url,is_ebook,...,publication_year,publisher,ratings_count,series,similar_books,text_reviews_count,title,title_without_series,url,work_id
0,,"[{'author_id': '15585', 'role': ''}]",3.83,16037549,US,Number 30 in a series of literary pamphlets pu...,,Paperback,https://images.gr-assets.com/books/1348176637m...,false,...,1887,"Houghton, Mifflin and Company",3,[],[],1,Vision of Sir Launfal and Other Poems,Vision of Sir Launfal and Other Poems,https://www.goodreads.com/book/show/16037549-v...,5212748
1,,"[{'author_id': '16073', 'role': ''}, {'author_...",3.83,22466716,US,Fairy Tales gathers the unconventional verse d...,,Paperback,https://images.gr-assets.com/books/1404958407m...,false,...,2015,New Directions,37,[],[],2,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes,https://www.goodreads.com/book/show/22466716-f...,41905435
2,,"[{'author_id': '18540', 'role': ''}, {'author_...",4.38,926662,US,Three poems describe the nighttime adventures ...,,Paperback,https://s.gr-assets.com/assets/nophoto/book/11...,false,...,2008,Farrar Straus Giroux,45,[],[],7,Growltiger's Last Stand and Other Poems,Growltiger's Last Stand and Other Poems,https://www.goodreads.com/book/show/926662.Gro...,911665
3,,"[{'author_id': '18540', 'role': ''}]",3.71,926667,US,A modern verse play about the search for meani...,,Paperback,https://images.gr-assets.com/books/1382939971m...,false,...,1964,Mariner Books,115,[],"[1230072, 315167, 676169, 18522, 124335, 88263...",12,The Cocktail Party,The Cocktail Party,https://www.goodreads.com/book/show/926667.The...,995066
4,,"[{'author_id': '14308759', 'role': ''}]",5.00,29065952,US,Louder Than Everything You Love is about trans...,First,Paperback,https://images.gr-assets.com/books/1455198396m...,false,...,2015,ELJ Publications,9,[],"[25869488, 23630890, 25448131, 25464039, 42166...",4,Louder Than Everything You Love,Louder Than Everything You Love,https://www.goodreads.com/book/show/29065952-l...,49294781


In [51]:
interaction = load_data('data/book/goodreads_interactions_poetry.json.gz')
interaction = pd.DataFrame(interaction)
interaction.head()

,book_id,date_added,date_updated,is_read,rating,read_at,review_id,review_text_incomplete,started_at,user_id
0,1384,Wed May 09 09:33:44 -0700 2007,Wed May 09 09:33:44 -0700 2007,True,4,,1bad0122cebb4aa9213f9fe1aa281f66,,,8842281e1d1347389f2ab93d60773d4d
1,1376,Wed May 09 09:33:18 -0700 2007,Wed May 09 09:33:18 -0700 2007,True,4,,eb6e502d0c04d57b43a5a02c21b64ab4,,,8842281e1d1347389f2ab93d60773d4d
2,30119,Sat Jan 13 13:44:20 -0800 2007,Wed Mar 22 11:45:08 -0700 2017,True,5,Tue Mar 01 00:00:00 -0800 1983,787564bef16cb1f43e0f641ab59d25b7,,,8842281e1d1347389f2ab93d60773d4d
3,24769928,Wed Apr 27 11:05:51 -0700 2016,Wed Apr 27 11:05:52 -0700 2016,False,0,,8c80ee74743d4b3b123dd1a2e0c0bcac,,,72fb0d0087d28c832f15776b0d936598
4,30119,Mon Jun 04 18:58:08 -0700 2012,Mon Jun 04 18:58:13 -0700 2012,True,3,,2a83589fb597309934ec9b1db5876aaf,,,72fb0d0087d28c832f15776b0d936598


In [54]:
# interaction = interaction[interaction.is_read == True]
item_unique = sorted(interaction.book_id.unique())
user_unique = sorted(interaction.user_id.unique())
print("# num of user: %d \n# num of item: %d"%(len(user_unique), len(item_unique)))

# num of user: 282414 
# num of item: 36373


In [ ]:
# Group data following user

def convert_time(t):
    d = t.split(" ")
    d = d[:4] + [d[-1]]
    d = " ".join(d)
    d = datetime.strptime(d, '%a %b %d %H:%M:%S %Y')
    unixtime = time.mktime(d.timetuple())
    return int(unixtime)


data_dir = "data/book/"
n_user = len(user_unique)
n_item = len(item_unique)

previous_user_id = 0
ratings = [0] * n_user
f = open("%s/ratings.txt"%data_dir, "w")
for _, r in interaction.iterrows():
    uid = user_unique.index(r.user_id)
    iid = item_unique.index(r.book_id)
    t = convert_time(r.date_added)
    if ratings[uid] == 0:
        ratings[uid] = [[iid, r.rating, t]]
    else:
        ratings[uid].append([iid, r.rating, t])
    f.write("%d::%i::%d::%s\n"%(uid, iid, int(r.rating), t))
f.close()

In [59]:
print("Max item user rated: %d"%max([len(i) for i in ratings]))
print("Min item user rated: %d"%min([len(i) for i in ratings]))
print("Mean item user rated: %d"%np.mean([len(i) for i in ratings]))

datetime.datetime(2007, 1, 13, 13, 44, 20)

In [44]:
# write new id of items into file
f = open("%s/item_id.txt"%data_dir, "w")
f.write("\n".join(item_unique))
f.close()

datetime.datetime(2007, 5, 9, 9, 33, 44)

In [ ]:
# write new id of users into file
f = open("%s/user_id.txt"%data_dir, "w")
f.write("\n".join(user_unique))
f.close()

In [ ]:
shuffle_id = np.random.permutation(n_user)
train_len = int(0.7*n_user)
train_id = shuffle_id[:train_len]
test_id = shuffle_id[train_len:]

ftrain = open("%s/train.txt"%data_dir, "w")
for idx in train_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftrain.write("%d %s\n"%(idx, " ".join(item)))
ftrain.close()

ftest = open("%s/test.txt"%data_dir, "w")
for idx in test_id:
    user = np.array(ratings[idx]).reshape((len(ratings[idx]), 3))
    user = user[np.argsort(user[:, 2])]
    item = list(user[:, 0])
    item = [str(i) for i in item]
    ftest.write("%d %s\n"%(idx, " ".join(item)))
ftest.close()